In [1]:
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error 

from kan import *
torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
def treasury_data_retrieval():
    df = pd.read_csv('../data/us_treasury_rates.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by='Date', ascending=True, inplace=True)
    df = df.reset_index(drop=True)

    return df

df = treasury_data_retrieval()

n = len(df)
h = 5

df_flat = pd.DataFrame()
for id in range(1, n):
    row = df.iloc[id:(id+h), 1:].stack().reset_index(drop=True).to_frame().T
    df_flat = pd.concat([df_flat, row], ignore_index=True)

for id in range(1, 13):
    df_flat[f'y_{id}'] = df.iloc[:(n-1), id]

df_flat['Date'] = df['Date']
df_flat.dropna(inplace=True)
df_flat.columns = df_flat.columns.astype(str)
df_flat.set_index('Date', inplace=True)

df_flat.tail()

,0,1,2,3,4,5,6,7,8,9,...,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,y_12
Date,,,,,,,,,,,,,,,,,,,,,
2024-11-22,4.74,4.67,4.62,4.46,4.37,4.21,4.21,4.17,4.21,4.27,...,4.63,4.46,4.42,4.37,4.32,4.30,4.35,4.41,4.67,4.60
2024-11-25,4.74,4.67,4.61,4.45,4.37,4.21,4.21,4.17,4.24,4.30,...,4.62,4.46,4.37,4.21,4.21,4.17,4.21,4.27,4.53,4.45
2024-11-26,4.76,4.70,4.60,4.43,4.34,4.19,4.17,4.11,4.17,4.25,...,4.61,4.45,4.37,4.21,4.21,4.17,4.24,4.30,4.56,4.48
2024-11-27,4.76,4.69,4.58,4.42,4.30,4.13,4.10,4.05,4.10,4.18,...,4.60,4.43,4.34,4.19,4.17,4.11,4.17,4.25,4.52,4.44
2024-11-29,4.75,4.63,4.51,4.43,4.30,4.17,4.11,4.08,4.13,4.19,...,4.58,4.42,4.30,4.13,4.10,4.05,4.10,4.18,4.45,4.36


In [3]:
def train_mse():
    predictions = model(dataset['train_input'])  # Model predictions
    mse = F.mse_loss(predictions, dataset['train_label'], reduction='mean')  # Compute MSE
    return mse  # Return scalar MSE value

def test_mse():
    predictions = model(dataset['test_input']) # Model predictions
    mse = F.mse_loss(predictions, dataset['test_label'], reduction='mean')  # Compute MSE
    return mse

In [6]:
import numpy as np

# Parameters for the sliding window
test_size = 10

# Store results for each fold
fold_results = {'train_mse': [], 'test_mse': []}

# Prepare data
X, y = df_flat.iloc[:, :-12], df_flat.iloc[:, -12:]
n_inputs = X.shape[1]
n_outputs = y.shape[1]

X_train, X_test = X[:-test_size], X[-test_size:]
y_train, y_test = y[:-test_size], y[-test_size:]

dataset = dict()
dtype = torch.get_default_dtype()
dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
dataset['train_label'] = torch.from_numpy(y_train.values).type(dtype).to(device)
dataset['test_input'] = torch.from_numpy(X_test.iloc[0, :].values.reshape(1, -1)).type(dtype).to(device)
dataset['test_label'] = torch.from_numpy(y_test.iloc[0, :].values.reshape(1, -1)).type(dtype).to(device)

# Initialize the model
model = KAN(width=[n_inputs, 20, n_outputs], grid=3, k=2, seed=42, device=device)

# Train the model and compute metrics
results = model.fit(dataset, opt="Adam", lamb=0.0, lr=0.001, steps=1000, metrics=(train_mse, test_mse))


feature = dataset['test_input']
output_list = list()
for id in range(1, test_size + 1):
    new = model(feature).cpu().detach().numpy().flatten()
    output_list.append(new)

    old = feature.cpu().detach().numpy().flatten()[(n_outputs):]
    feature = torch.from_numpy(np.append(old, new).reshape(1, -1)).type(dtype).to(device)
    
    
# Store the metrics
fold_results['train_mse'].append(results['train_mse'][-1])
fold_results['test_mse'].append(results['test_mse'][-1])

# Calculate average metrics across all windows
avg_train_mse = np.mean(fold_results['train_mse'])
avg_test_mse = np.mean(fold_results['test_mse'])

print("Sliding Window Cross-Validation Results")
print(f"Average Train MSE: {avg_train_mse}")
print(f"Average Test MSE: {mean_squared_error(output_list, y_test)}")

checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                         | 0/1000 [00:00<?, ?it/s]

| train_loss: 9.02e-02 | test_loss: 4.13e-02 | reg: 0.00e+00 | : 100%|▉| 997/1000 [00:50<00:00, 20.4/usr/local/python/3.12.1/lib/python3.12/site-packages/kan/MultKAN.py:813: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1808.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
/usr/local/python/3.12.1/lib/python3.12/site-packages/kan/MultKAN.py:823: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1808.)
  input_range = torch.std(preacts, dim=0) + 0.1
/usr/local/python/3.12.1/lib/python3.12/site-packages/kan/MultKAN.py:824: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by 

saving model version 0.1
Sliding Window Cross-Validation Results
Average Train MSE: 0.008116570313334799
Average Test MSE: 0.0033573600730667954


In [7]:
df_naive = pd.DataFrame([y_train.iloc[-1]] * test_size, columns=y_train.columns)
mean_squared_error(df_naive, y_test)
# df_naive

np.float64(0.007895000000000004)

In [8]:
pd.DataFrame(output_list)

,0,1,2,3,4,5,6,7,8,9,10,11
0,4.704344,4.658192,4.634838,4.447138,4.393315,4.266103,4.242995,4.263711,4.321331,4.394254,4.615909,4.575229
1,4.717122,4.650625,4.637676,4.456458,4.383910,4.260013,4.239261,4.252709,4.316193,4.382722,4.621207,4.565418
2,4.696376,4.662907,4.654073,4.453911,4.404207,4.278262,4.245660,4.258255,4.319450,4.388363,4.601996,4.560436
3,4.692003,4.657624,4.660989,4.439422,4.412810,4.285480,4.255715,4.262527,4.321174,4.391696,4.597501,4.556609
4,4.699693,4.658849,4.652656,4.454937,4.409619,4.273875,4.255895,4.260749,4.309259,4.383428,4.613072,4.557582
5,4.722978,4.668385,4.646367,4.467094,4.375419,4.237214,4.221150,4.232764,4.284279,4.361922,4.612668,4.545167
6,4.722968,4.668302,4.647167,4.470728,4.374830,4.238134,4.218592,4.224378,4.277291,4.354834,4.605758,4.536216
7,4.709590,4.670974,4.654517,4.474848,4.380404,4.241557,4.217819,4.222287,4.273648,4.354235,4.599241,4.529109
8,4.707395,4.673118,4.658946,4.473976,4.382732,4.238224,4.216466,4.222207,4.270950,4.353186,4.598419,4.528543
9,4.722271,4.679645,4.660369,4.475284,4.373370,4.222874,4.204846,4.214574,4.265641,4.347505,4.598349,4.530811


In [9]:
y_test

,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9,y_10,y_11,y_12
Date,,,,,,,,,,,,
2024-11-15,4.70,4.67,4.60,4.44,4.34,4.31,4.27,4.30,4.36,4.43,4.70,4.60
2024-11-18,4.70,4.65,4.63,4.44,4.33,4.29,4.25,4.28,4.35,4.42,4.70,4.61
2024-11-19,4.67,4.63,4.62,4.44,4.34,4.27,4.24,4.25,4.32,4.39,4.66,4.57
2024-11-20,4.68,4.63,4.62,4.44,4.37,4.31,4.26,4.28,4.34,4.41,4.66,4.59
2024-11-21,4.72,4.67,4.63,4.45,4.39,4.34,4.30,4.30,4.36,4.43,4.68,4.61
2024-11-22,4.72,4.67,4.63,4.46,4.42,4.37,4.32,4.30,4.35,4.41,4.67,4.60
2024-11-25,4.74,4.67,4.62,4.46,4.37,4.21,4.21,4.17,4.21,4.27,4.53,4.45
2024-11-26,4.74,4.67,4.61,4.45,4.37,4.21,4.21,4.17,4.24,4.30,4.56,4.48
2024-11-27,4.76,4.70,4.60,4.43,4.34,4.19,4.17,4.11,4.17,4.25,4.52,4.44
